# Basic Routing with OpenRouteService

Calculate routes and distances from origins and destinations stored in separate CSV files with coordinates, and generate shapefiles and plot on a Folium Map. Input CSVs must contain a header row, with attributes that include a unique id, name, longitude, and latitude in WGS 84

Using https://openrouteservice.org/ and https://pypi.org/project/openrouteservice/ with basic examples: https://openrouteservice-py.readthedocs.io/en/latest/

Frank Donnelly / Head GIS and Data Services / Brown University Library
May 31, 2024 | Revised Mar 2, 2025

## Brown Univ CoLab Users¶

1. Open this notebook with this URL: https://colab.research.google.com/github/Brown-University-Library/geodata_routing/blob/main/routing_openroute.ipynb

2. Run the following two boxes to import this repo into a temporary folder, and install the openrouteservice module


In [ ]:
# GOOGLE COLAB USERS - RUN THIS
!git clone https://github.com/Brown-University-Library/geodata_routing temp_repo && mv temp_repo/* temp_repo/.[!.]* . && rm -rf temp_repo

In [ ]:
# GOOGLE COLAB USERS - RUN THIS
!pip install openrouteservice

## Sign up and Request API Key

1. Register for an API Key at: https://account.heigit.org/signup
2. Create a text file called **ors_key.txt** in the root folder (where the scripts are stored), and copy / paste / save the key in the file

## Import Modules

In [ ]:
import openrouteservice, os, csv, pandas as pd, geopandas as gpd, folium
from shapely.geometry import shape
from openrouteservice.directions import directions
from openrouteservice import convert
from datetime import date

## Variables

In [ ]:
# general description, used in output file
routename='scili_to_libs'
# transit modes: [“driving-car”, “driving-hgv”, “foot-walking”, “foot-hiking”, “cycling-regular”, “cycling-road”,”cycling-mountain”, “cycling-electric”,]
tmode='driving-car'
# distance units: [“m”, “km”, “mi”]
dunits='mi'
# route preference: [“fastest, “shortest”, “recommended”]
rpref='fastest'

In [ ]:
# Origin CSV: Column positions that contain: unique ID, name, longitude, latitude
ogn_id=0
ogn_name=1
ogn_long=2
ogn_lat=3

In [ ]:
# Destination CSV: Column positions that contain: unique ID, name, longitude, latitude
d_id=0
d_name=1
d_long=2
d_lat=3

In [ ]:
# INPUT Files - Modify to specify input

today=str(date.today()).replace('-','_')

keyfile='ors_key.txt'
origin_file=os.path.join('input','origins.csv')
dest_file=os.path.join('input','destinations.csv')

In [ ]:
# OUTPUT Files

route_file=routename+'_'+tmode+'_'+rpref+'_'+today+'.shp'
out_file=os.path.join('output',route_file)
out_origin=os.path.join('output',os.path.basename(origin_file).split('.')[0]+'_'+today+'.shp')
out_dest=os.path.join('output',os.path.basename(dest_file).split('.')[0]+'_'+today+'.shp')

## Functions

In [ ]:
# For reading origin and dest files
def file_reader(infile,outlist):
    with open(infile,'r') as f:
        reader = csv.reader(f)    
        for row in reader:
            rec = [i.strip() for i in row]
            outlist.append(rec)

In [ ]:
# For converting origins and destinations to geodataframes            
def coords_to_gdf(data_list,long,lat,export):
    """Provide: list of places that includes a header row,
    positions in list that have longitude and latitude, and
    path for output file.
    """
    df = pd.DataFrame(data_list[1:], columns=data_list[0])
    longcol=data_list[0][long]
    latcol=data_list[0][lat]
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[longcol], df[latcol]), crs='EPSG:4326')
    gdf.to_file(export,index=True)
    print('Wrote shapefile',export,'\n')
    return(gdf)

## Read Files

In [ ]:
origins=[]
file_reader(origin_file,origins)
origins

In [ ]:
dest=[]
file_reader(dest_file,dest)
dest

In [ ]:
route_count=0
route_list=[]
# Column header for route output file:
header=['ogn_id','ogn_name','dest_id','dest_name','distance','travtime','route']

In [ ]:
# Read api key in from file
with open(keyfile) as key:
    api_key=key.read().strip()

# REQUESTS BLOCK - API CALL

In [ ]:
for ogn in origins[1:]:
    for d in dest[1:]:
        try:
            coords=((ogn[ogn_long],ogn[ogn_lat]),(d[d_long],d[d_lat]))
            client = openrouteservice.Client(key=api_key) 
            # Take the returned object, save into nested dicts:
            results = directions(client, coords, 
                                profile=tmode,instructions=False, preference=rpref,units=dunits)
            dist = results['routes'][0]['summary']['distance']
            travtime=results['routes'][0]['summary']['duration']/60 # Get minutes
            encoded_geom = results['routes'][0]['geometry']
            decoded_geom = convert.decode_polyline(encoded_geom) #convert from binary to txt
            wkt_geom=shape(decoded_geom).wkt #convert from json polyline to wkt
            route=[ogn[ogn_id],ogn[ogn_name],d[d_id],d[d_name],dist,travtime,wkt_geom]
            route_list.append(route)
            route_count=route_count+1
            if route_count%40==0: # API limit is 40 requests per minute
                print('Pausing 1 minute, processed',route_count,'records...')
                sleep(60)
        except Exception as e:
            print(str(e))

print('Plotted',route_count,'routes...' )
api_key=''

In [ ]:
# For example: original output record
results

In [ ]:
# For example: extracted and formatted output record
route

## Generate Output

In [ ]:
# Create shapefiles for origins and destinations
ogdf=coords_to_gdf(origins,ogn_long,ogn_lat,out_origin)
dgdf=coords_to_gdf(dest,d_long,d_lat,out_dest)

In [ ]:
# Create dataframes and shapefile
df = pd.DataFrame(route_list, columns=header)
gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df["route"]),crs = 'EPSG:4326')
gdf.drop(['route'],axis=1, inplace=True) # drop the wkt text
gdf.to_file(out_file,index=True) # writes out a shapefile
print("Wrote routes shapefile to:", out_file)
gdf


In [ ]:
# Get colors for lines
gdfcount=len(gdf) # number of routes
colors=['red','green','blue','gray','purple','brown']
clist=[] # list of colors, one per route
c=0
for i in range(gdfcount):
    clist.append(colors[c])
    c=c+1
    if c>len(colors)-1:
        c=0 # if we run out of colors, start over
color_series = pd.Series(clist,name='color') # create series in order to...
gdf_c=pd.merge(gdf, color_series, left_index=True,right_index=True) # join to routes on seq index #
gdf_c

In [ ]:
# Create folium map: https://python-visualization.github.io/folium/latest/
# get center point of bbox to specify location for folium
bnds=gdf.total_bounds 
clong=(bnds[0]+bnds[2])/2
clat=(bnds[1]+bnds[3])/2

m = folium.Map(location=[clat,clong], tiles="OpenStreetMap")
popup = folium.GeoJsonPopup(
    fields=["ogn_name", "dest_name","distance","travtime"],
    localize=True,
    labels=True)
folium.GeoJson(gdf_c,style_function=lambda x: {'color':x['properties']['color']},popup=popup).add_to(m)

folium.GeoJson(ogdf,marker=folium.Marker(icon=folium.Icon(icon='home',color='black'))).add_to(m)
folium.GeoJson(dgdf,marker=folium.Marker(icon=folium.Icon(icon='star',color='lightgray'))).add_to(m)
m.fit_bounds(m.get_bounds()) # zoom to bounding box
m

To see the Folium map in GitHub, view this notebook in nbviewer:

https://nbviewer.org/github/Brown-University-Library/geodata_routing/blob/main/routing_openroute.ipynb